In [1]:
import numpy as np 
import crocoddyl 
import matplotlib.pyplot as plt 
#
import os, sys
src_path = os.path.abspath('../../') 
sys.path.append(src_path) 
#
from utils.action_models import penumatic_hopper 
from solvers import furisc
from utils.uncertainty import measurement_models, process_models, problem_uncertainty
from utils.problems import penumatic_hopper_problem 
from hopper_config import *

In [2]:
# setup optimal control problem and solver 
# horizon = 100
# uncertainty models 
initial_covariance = 1.e-3 * np.eye(4)
process_noise = 1.e-4*np.eye(4)
measurement_noise = 1.e-5*np.eye(4)
sensitivity = -.01
MAX_ITER = 20

pmodels, umodels = penumatic_hopper_problem.full_state_uniform_hopper(dt, horizon, process_noise, measurement_noise)

problem = crocoddyl.ShootingProblem(x0, pmodels[:-1], pmodels[-1])

irisc_uncertainty = problem_uncertainty.ProblemUncertainty(x0, initial_covariance, umodels)
solver = furisc.FeasibilityRiskSensitiveSolver(problem, irisc_uncertainty, sensitivity)

solver.setCallbacks([
crocoddyl.CallbackLogger(),
crocoddyl.CallbackVerbose()
])



xs_ddp = np.load("solutions/ddp_xs.npy")
us_ddp = np.load("solutions/ddp_us.npy")
xs = [xi.copy() for xi in xs_ddp]
us = [ui.copy() for ui in us_ddp]

# xs = [x0]*(horizon+1)
# us = [np.array([0.])]*horizon    

In [ ]:
solver.solve(xs, us, maxiter=MAX_ITER, isFeasible=False)

initial cost is 0.006782943337958283
initial trajectory feasibility False


In [ ]:
# # Desparate Attempt to solve this issue 

# solver.setCandidate(xs, us, False)
# solver.cost = solver.unscentedCost()
# print("initial cost is %s"%solver.cost)
# print("initial trajectory feasibility %s"%solver.isFeasible)      
# solver.n_little_improvement = 0
# solver.x_reg = solver.regMin
# solver.u_reg = solver.regMin
# # computeDirection 
# solver.computeDirection(True)

# xf, uf, cost = solver.unscentedForwardPass(1.)

# costs = []
# for i in range(solver.sample_size):
#     if i == 0:
#         costs += [solver.w0 * np.exp(-solver.sigma *np.sum(solver.sample_costs[:,i])) ]
#     else:
#         costs += [solver.wi * np.exp(-solver.sigma *np.sum(solver.sample_costs[:,i])) ]
# print(sum(costs))



# ctry =  - solver.inv_sigma*np.log(sum(costs))

# print("cost try =", ctry)
# print("solver.w0 =", solver.w0)
# print("solver.wi =",solver.wi)
# print("solver.rv_dim =",solver.rv_dim)
# print("solver.sample_size =", solver.sample_size)
# print("solver.lam =",solver.lam)

In [ ]:
solver.sample

In [ ]:
costs = []
for i in range(solver.sample_size):
    if i == 0:
        costs += [solver.w0 * np.exp(-solver.sigma *np.sum(solver.sample_costs[:,i])) ]
    else:
        costs += [solver.wi * np.exp(-solver.sigma *np.sum(solver.sample_costs[:,i])) ]

In [ ]:
print(sum(costs))

In [ ]:
np.log(5.529976935228933e-27)